# SETUP

In [21]:
# TA-LIB
!wget https://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
!tar -xzf ta-lib-0.4.0-src.tar.gz
%cd ta-lib/
!./configure --prefix=/usr
!make
!sudo make install
%cd /content


--2022-09-07 19:42:55--  https://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Connecting to downloads.sourceforge.net (downloads.sourceforge.net)|204.68.111.105|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://altushost-swe.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2022-09-07 19:42:57--  https://altushost-swe.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving altushost-swe.dl.sourceforge.net (altushost-swe.dl.sourceforge.net)... 79.142.76.130
Connecting to altushost-swe.dl.sourceforge.net (altushost-swe.dl.sourceforge.net)|79.142.76.130|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1330299 (1.3M) [application/x-gzip]
Saving to: ‘ta-lib-0.4.0-src.tar.gz’

ta-lib-0.4.0-src.ta   6%[>                   ]  79.76K  --.-KB/s    in 81s     

20

In [22]:

#
!pip uninstall -y matplotlib imgaug
!pip install backtrader matplotlib==3.2.2 seaborn TA-Lib hyperopt backtrader-plotting

^C
Traceback (most recent call last):
  File "/workspace/habr/jeezy/.venv/bin/pip", line 5, in <module>
    from pip._internal.cli.main import main
  File "/workspace/habr/jeezy/.venv/lib/python3.9/site-packages/pip/_internal/cli/main.py", line 9, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/workspace/habr/jeezy/.venv/lib/python3.9/site-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/workspace/habr/jeezy/.venv/lib/python3.9/site-packages/pip/_internal/cli/main_parser.py", line 8, in <module>
    from pip._internal.cli import cmdoptions
  File "/workspace/habr/jeezy/.venv/lib/python3.9/site-packages/pip/_internal/cli/cmdoptions.py", line 23, in <module>
    from pip._internal.cli.parser import ConfigOptionParser
  File "/workspace/habr/jeezy/.venv/lib/python3.9/site-packages/pip/_internal/cli/parser.py", line 12, in <module>
    from pip._internal.configura

In [5]:
import timeit
import sys
import datetime
import time
import re
import glob
from functools import partial

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import pandas as pd

import backtrader as bt  # Import the backtrader platform
from backtrader_plotting import Bokeh

import requests
import json
import numpy as np

IS_IN_COLAB = 'google.colab' in sys.modules


# BASE PARAMETERS


In [28]:
# BASE PARAMETERS



In [6]:
#@title TICKET AND DATAFRAME
# DODOUSDTPERP
TICKER = "BTCUSDT" #@param {type:"string"}
TIME_FRAME = "15m" #@param {type:"string"}
FROM_DATE = "2018-08-01" #@param {type:"string"}
ITERATIONS = 100 #@param {type:"integer"}
#@markdown Commision (%) (default: 0.1%)
COMMISSION_PRST = 0.1 #@param {type:"number"}
COMMISSION = COMMISSION_PRST / 100


# GET DATA


In [7]:

# init data

FN_TICKERS_REGEXP = f'({TICKER}\,\s*{TIME_FRAME}\.csv$)|({TICKER}_{TIME_FRAME}.csv$)'
all_dir = glob.glob('./*.csv', recursive=False) + glob.glob('./data/*.csv', recursive=False)
files_for_ticker = [fn for fn in all_dir if re.search(FN_TICKERS_REGEXP, fn, re.I)]
is_data_has_volume = True

max_val = 0

if files_for_ticker:
    datapath = files_for_ticker[0]
    df = pd.read_csv(datapath, header=0, encoding='utf-8', sep=',')
    df['time'] = pd.to_datetime(df['time'], unit='s')
    is_data_has_volume = 'volume' in df.columns
    df_filtered = df[df['time'] > pd.to_datetime(FROM_DATE)]
    df = df_filtered.reset_index().set_index('time')
    # max_val = df['high'].max()
    # data = TVPandasData(dataname=df, name=TICKER.lower())
else:
    from_date_val = datetime.datetime.fromisoformat(FROM_DATE)
    start_time = int(from_date_val.timestamp() * 1000)

    df = pd.DataFrame(columns=['time', 'open', 'high', 'low', 'close', 'volume'])
    opentime, lopen, lhigh, llow, lclose, lvol, closetime = [], [], [], [], [], [], []

    cur_time = datetime.datetime.utcnow().replace(minute=0, second=0, microsecond=0)
    # cur_time = cur_time.replace(minute=0, second=0, microsecond=0)
    cur_time = int(cur_time.timestamp()) * 1000 - 1

    if 'perp' in TICKER.lower():
        api_url = f'https://fapi.binance.com/fapi/v1/continuousKlines?pair={TICKER[:-len("perp")]}&interval={TIME_FRAME.lower()}&contractType=PERPETUAL&limit=1000'
    else:
        api_url = f'https://fapi.binance.com/fapi/v1/klines?symbol={TICKER}&interval={TIME_FRAME.lower()}&limit=1000'
    while start_time < cur_time:
        print(f'Get 1000 candles for {TICKER}:{TIME_FRAME} from the datetime {datetime.datetime.utcfromtimestamp(int(start_time / 1000))}')
        sym_info = requests.get(f"{api_url}&startTime={start_time}")
        if (sym_info.status_code == 429):
            raise InterruptedError(f'Limits is violated')
        elif (sym_info.status_code != 200):
            raise InterruptedError(f'Error for request. Code {sym_info.status_code}')
        candles = json.loads(sym_info.text)
        for candle in candles:
            opentime.append(candle[0])
            lopen.append(candle[1])
            lhigh.append(candle[2])
            llow.append(candle[3])
            lclose.append(candle[4])
            lvol.append(candle[5])
            closetime.append(candle[6])
        start_time = closetime[-1]
        time.sleep(0.25)

    df['open_time'] = pd.to_datetime(opentime, unit='ms', origin='unix')
    df['time'] = df['open_time'].apply(lambda x: int(x.timestamp()))
    df['open'] = np.array(lopen).astype(float)
    df['high'] = np.array(lhigh).astype(float)
    df['low'] = np.array(llow).astype(float)
    df['close'] = np.array(lclose).astype(float)
    df['volume'] = np.array(lvol).astype(float)
    df.to_csv(f'./{TICKER}, {TIME_FRAME}.csv', header=True, index=False, date_format='%s', encoding='utf-8',
              sep=',')
    df['time'] = df['open_time']
    df.set_index('time', inplace=True)
    # max_val = df['high'].max()
    is_data_has_volume = 'volume' in df.columns
    # data = TVPandasData(dataname=df, name=TICKER.lower())

max_val = df['high'].max()
class TVPandasData(bt.feeds.PandasData):
    params = (
        ('datetime', None),
        ('open', 'open'),
        ('high', 'high'),
        ('low', 'low'),
        ('close', 'close'),
        ('volume', 'volume' if is_data_has_volume else None),
        ('openinterest', None),
    )
data = TVPandasData(dataname=df, name=TICKER.lower())



Get 1000 candles for BTCUSDT:15m from the datetime 2018-07-31 14:00:00
Get 1000 candles for BTCUSDT:15m from the datetime 2019-09-19 03:44:59
Get 1000 candles for BTCUSDT:15m from the datetime 2019-09-29 13:44:59
Get 1000 candles for BTCUSDT:15m from the datetime 2019-10-09 23:44:59
Get 1000 candles for BTCUSDT:15m from the datetime 2019-10-20 09:44:59
Get 1000 candles for BTCUSDT:15m from the datetime 2019-10-30 19:44:59
Get 1000 candles for BTCUSDT:15m from the datetime 2019-11-10 05:44:59
Get 1000 candles for BTCUSDT:15m from the datetime 2019-11-20 15:44:59
Get 1000 candles for BTCUSDT:15m from the datetime 2019-12-01 01:44:59
Get 1000 candles for BTCUSDT:15m from the datetime 2019-12-11 11:44:59
Get 1000 candles for BTCUSDT:15m from the datetime 2019-12-21 21:44:59
Get 1000 candles for BTCUSDT:15m from the datetime 2020-01-01 07:44:59
Get 1000 candles for BTCUSDT:15m from the datetime 2020-01-11 17:44:59
Get 1000 candles for BTCUSDT:15m from the datetime 2020-01-22 03:44:59
Get 10

# Base strategy parameters


In [31]:

base_param = {
    'stake': 2, #  Calculate from persentage of LongQtyPrcnt, Now it 50%
    'commission': COMMISSION,
    'money_start': max_val * 10 if max_val != 0 else 1000000,
    'slip_perc': 0.005, # TODO Slippage https://www.backtrader.com/docu/slippage/slippage/
    'StartDateTime': FROM_DATE,  # Start Time
    'EndDateTime': None  # End Time
}


# Strategy parameters
# Strategy

In [32]:
class MAcrossover(bt.Strategy):
    # Moving average parameters
    params = (('pfast', 10), ('pslow', 20),)

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        # print(f'{len(self)} {dt.isoformat()} {txt}')  # Comment this line when running optimization

    def __init__(self):
        self.dataclose = self.datas[0].close

        # Order variable will contain ongoing order details/status
        self.order = None

        # Instantiate moving averages
        self.slow_sma = bt.ind.MovingAverageSimple(self.datas[0],
                                                   period=self.params.pslow)
        self.fast_sma = bt.ind.MovingAverageSimple(self.datas[0],
                                                   period=self.params.pfast)

        self.crossover = bt.ind.CrossOver(self.fast_sma, self.slow_sma)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # An active Buy/Sell order has been submitted/accepted - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, {order.executed.price:.2f}')
            elif order.issell():
                self.log(f'SELL EXECUTED, {order.executed.price:.2f}')
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Reset orders
        self.order = None

    def next(self):
        # Check for open orders
        if self.order:
            return

        # Check if we are in the market
        if not self.position:
            # We are not in the market, look for a signal to OPEN trades

            # If the 20 SMA is above the 50 SMA
            if self.crossover > 0:  # Fast ma crosses above slow ma
                # if self.fast_sma[0] > self.slow_sma[0] and self.fast_sma[-1] < self.slow_sma[-1]:
                self.log(f'BUY CREATE {self.dataclose[0]:2f}')
                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()
            # Otherwise if the 20 SMA is below the 50 SMA
            elif self.crossover < 0:  # Fast ma crosses below slow ma
                # elif self.fast_sma[0] < self.slow_sma[0] and self.fast_sma[-1] > self.slow_sma[-1]:
                self.log(f'SELL CREATE {self.dataclose[0]:2f}')
                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()
        else:
            # We are already in the market, look for a signal to CLOSE trades
            if len(self) >= (self.bar_executed + 5):
                self.log(f'CLOSE CREATE {self.dataclose[0]:2f}')
                self.order = self.close()

    def stop(self):
        self.log(
            f'(MA Period fast: {self.params.pfast}, MA Period slow: {self.params.pslow}  Ending Value {self.broker.getvalue()}')




# Backtester functions


In [33]:

def get_cerebro(data, base_param, vp):
    cerebro = bt.Cerebro(optreturn=True, preload=True, runonce=True)
    cerebro.adddata(data)

    # TODO only need analyzer that used for strategy optimization(!)
    cerebro.addsizer(bt.sizers.FixedSize, stake=base_param['stake'])
    # cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')
    cerebro.addstrategy(MAcrossover, **vp)
    cerebro.broker = bt.brokers.BackBroker(cash=base_param['money_start'],
                                           slip_perc=base_param['slip_perc'])  # https://www.backtrader.com/docu/slippage/slippage/
    cerebro.broker.setcommission(commission=base_param['commission'])

    return cerebro

def runStrategy(data, base_param, vp):
    cerebro = get_cerebro(data, base_param, vp)
    stats = cerebro.run()
    net_profit = round(cerebro.broker.get_value() - base_param['money_start'], 2)
    # sharp =  stats[0].analyzers.sharpe_ratio.get_analysis() # -(stats[0].analyzers.sharpe_ratio.get_analysis()),
    # cerebro.plot(style='candlestick')

    # sharpe = stats[0].analyzers.sharpe_ratio.get_analysis()
    # drawdown = stats[0].analyzers.drawdown.get_analysis()
    trades = stats[0].analyzers.trades.get_analysis()
    # AnnualReturn = strategy.analyzers.AnnualReturn.get_analysis() # Calmar = strategy.analyzers.Calmar.get_analysis() # DrawDown = strategy.analyzers.DrawDown.get_analysis()
    stats_res = {
        'net_profit': net_profit,
        'profit': cerebro.broker.get_value(),
        'profit_rate': round(net_profit / base_param['money_start'] * 100, 2),
        # 'sharpe': sharpe['sharperatio'],
        # 'drawdown': drawdown['drawdown'], # max.moneydown # https://www.backtrader.com/docu/analyzers-reference/#drawdown
        # 'drawdown_money': drawdown['moneydown'],
        # 'drawdown_len': drawdown['len'],
        # 'drawdown_max': drawdown['max']['drawdown'],
        # 'drawdown_max_money': drawdown['max']['moneydown'],
        # 'trades_total': stats[0].wintrades[-1] + stats[0].losstrades[-1] + stats[0].eventrades[-1],
        # 'trades_win': stats[0].wintrades[-1],
        # 'trades_loss': stats[0].losstrades[-1] + stats[0].eventrades[-1]
        'trades_total': trades['total']['total'] # https://www.backtrader.com/docu/analyzers-reference/#tradeanalyzer
        }
    del cerebro
    del stats

    return {
        # 'loss': -(net_profit - drawdown['max']['moneydown']) if type(net_profit) != type(None) else base_param['money_start'],
        'stats': stats_res,
        'params': vp
    }


# RUN STRATEGY


In [34]:
best_param = ({'pfast': 9, 'pslow': 21})


partialRunStrategy = partial(runStrategy, data, base_param)
# cycles = 5
# rest_time=timeit.timeit(lambda: runStrategy(data, base_param, best_param), number=cycles)
# print(rest_time, rest_time/cycles)
res = partialRunStrategy(best_param)

print(res)

{'stats': {'net_profit': -19212.5, 'profit': 232787.4982, 'profit_rate': -7.62, 'trades_total': 185}, 'params': {'pfast': 9, 'pslow': 21}}


In [35]:
#%matplotlib inline
# Draw the best parameters strategy results

cerebro = get_cerebro(data, base_param, best_param)
stats = cerebro.run()
if IS_IN_COLAB:
    b = Bokeh(style='bar', plot_mode='single')
    cerebro.plot(b, volume=is_data_has_volume)
else:
    cerebro.plot(style='candlestick', iplot=False)



ImportError: cannot import name 'warnings' from 'matplotlib.dates' (/workspace/habr/jeezy/.venv/lib/python3.9/site-packages/matplotlib/dates.py)